# Xây dựng hệ thống response với câu trả lời của khách.

Các bước sẽ thực hiện.

1 - tải các thư viện cần thiết bên dưới.

2 - tải model phobert đã được train sẵn trước đó.

3 - Tiền xử lý comment: loại bỏ các thành phần ảnh hưởng đến kết quả phân loại như: emoj, viết tắt, các dấu câu, ...

4 - Dùng model đã tải để đưa ra loại cảm xúc.

5 - dùng `all-MiniLM-L6-v2` để tính độ tương đồng đưa ra response hợp lý.

**Lưu ý:** mô hình phobert sau khi được train được lưu trên hugging face: "https://huggingface.co/phucgiacat/sentiment-vietnamese-phobert"

## 1. Tải các thư viện cần thiết

In [1]:
!pip install transformers
!pip install tf-keras
!pip install sentence-transformers
!pip install huggingface_hub

## 2. Tải model phoBert đã được train trước.

In [2]:
from transformers import pipeline
from sentence_transformers import SentenceTransformer, util
from sentence_transformers.util import cos_sim
import torch

model = SentenceTransformer('all-MiniLM-L6-v2') 
sentiment_pipeline = pipeline("text-classification", model="phucgiacat/sentiment-vietnamese-phobert", tokenizer="phucgiacat/sentiment-vietnamese-phobert")

/home/xv6/miniconda3/envs/ml-env/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-04-28 20:15:52.938089: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-04-28 20:15:53.297671: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1745871353.432377     397 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1745871353.470426     397 cuda_blas.cc:140

## 3.  Tìm response để phản hồi hợp lý

In [3]:
import json
import teencode_mean

with open('data/response_data.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

def find_best_response(user_message, data, predeicted_emotion):
    samples = data.get(predeicted_emotion, [])
    if not samples:
        return "Xin lỗi mình chưa tìm thấy thông tin phù hợp. Mình sẽ chuyển bạn đến nhân viên hỗ trợ nhé."
    user_embedding = model.encode(user_message, convert_to_tensor=True)
    sample_embeddings = model.encode([s["text"] for s in samples], convert_to_tensor=True)
    cosine_scores = util.cos_sim(user_embedding, sample_embeddings)
    if torch.max(cosine_scores) < 0.3:
        return "Bạn có thể nói rõ để hệ thống có thể hỗ trợ mình một cách tốt nhất ạ !"
    best_match_idx = torch.argmax(cosine_scores).item()
    return samples[best_match_idx]["response"]

## 4. Xử lý comment bỏ đi các thành phần gây nhiễu.

In [4]:
from underthesea import word_tokenize
from dataclasses import dataclass  # <-- đúng chuẩn
import re
import teencode_mean

@dataclass
class preProcess:
    emoji_pattern = re.compile("[" 
                u"\U0001F600-\U0001F64F"
                u"\U0001F300-\U0001F5FF"
                u"\U0001F680-\U0001F6FF"
                u"\U0001F1E0-\U0001F1FF"
                u"\U00002702-\U000027B0"
                u"\U000024C2-\U0001F251"
                u"\U0001f926-\U0001f937"
                u'\U00010000-\U0010ffff'
                u"\u200d"
                u"\u2640-\u2642"
                u"\u2600-\u2B55"
                u"\u23cf"
                u"\u23e9"
                u"\u231a"
                u"\u3030"
                u"\ufe0f"
    "]+", flags=re.UNICODE)

    def text_lower(self, text):
        return text.lower()

    def remove_numbers(self, text):
        return ''.join([i for i in text if not i.isdigit()])

    def remove_punctuation(self, text):
        text = text.replace(",", " ").replace(".", " ") \
            .replace(";", " ").replace("“", " ") \
            .replace(":", " ").replace("”", " ") \
            .replace('"', " ").replace("'", " ") \
            .replace("!", " ").replace("?", " ") \
            .replace("-", " ").replace("?", " ")
        return text

    def remove_extra_whitespace(self, text):
        return ' '.join(text.split())

    def remove_repeated_characters(self, text):
        return ''.join([text[i] for i in range(len(text)) if i == 0 or text[i] != text[i-1]])

    def remove_special_characters(self, text):
        return ''.join([i for i in text if i.isalnum() or i.isspace()])

    def VN_tokenize(self, text):
        return word_tokenize(text, format="text")

    def Util(self, text):
        text = text.split()
        length = len(text)
        for i in range(length):
            if text[i] in list(teencode_mean.teencodes.keys()):
                text[i] = teencode_mean.teencodes[text[i]]
        return ' '.join(text)

    def Text_preProcessing(self, text):
        text = self.text_lower(text)
        text = re.sub(self.emoji_pattern, " ", text)
        text = self.remove_punctuation(text)
        text = self.remove_extra_whitespace(text)
        text = self.remove_repeated_characters(text)
        text = self.Util(text)
        text = self.remove_numbers(text)
        return text

# Test thử
process_text = preProcess()

## 3. đưa ra kết quả cuối cùng.

In [6]:
id2label = {0: "POS", 1: "NEU", 2: "NEG"}
comment = "Sản phầm xài cảm giác rất tốt"
comment = process_text.Text_preProcessing(comment)
result = sentiment_pipeline(comment)
label_id = int(result[0]['label'].split('_')[-1])
predict = id2label[label_id]

best_response = find_best_response(comment, data, predict)
print(f"comment đã xử lý: {comment}")
print(f"loại cảm xúc comment: {predict}")
print(f"respone: {best_response}")

comment đã xử lý: sản phầm xài cảm giác rất tốt
loại cảm xúc comment: POS
respone: WOODA cảm ơn bạn rất nhiều ạ! Nếu bạn cần thêm sản phẩm mới, cứ nhắn mình nhé 💖.


sẽ còn phát triển thêm ~~ cố lên nha mọi người :)))